In [1]:
import os
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd


def manual_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    # if you are suing GPU
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


def get_model_size(model):
	total_size = sum(param.numel() for param in model.parameters() if param.requires_grad)
	return total_size / 1e6

manual_seed()


# Data

In [2]:
from utils import load_data, load_edge

train_folds = load_data(True)
test_fold = load_data(False)[0]

In [3]:
import torch
from torch.utils.data import DataLoader, Dataset

class SimpleStockDataset(Dataset):
    def __init__(self, data, ws=128):
        self.data = data
        self.ws = ws
        self.samples = []
        
        self.n_tickers, self.n_days, self.n_features = self.data.shape
        
        for start in range(self.n_days - self.ws + 1):
            self.samples.append(start)
            
    def __len__(self):
        return len(self.samples)
      
    def __getitem__(self, idx):
        start = self.samples[idx]
        x = torch.tensor(self.data[:, start:start + self.ws], dtype=torch.float32)
        return x
    

# Model

### LSTM

In [4]:

from __future__ import annotations
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
import torch.nn as nn
import torch.nn.functional as F


class MyRNN(nn.Module):
    def __init__(self, n_nodes: int, n_feats: int, args, node_emb=None):
        super().__init__()
        self.n_nodes = n_nodes
        self.rank_A = 32
        self.n_feats = n_feats
        self.args = args
        self.d_latent = getattr(args, "d_latent", 128)
        self.dropout = getattr(args, "dropout", 0.0)
        self.n_layers = getattr(args, "n_layers", 1)
        self.d_node = getattr(args, "d_node", 32)
        
        

        if node_emb is None:
            self.static_node_features = nn.Parameter(torch.randn(n_nodes, self.d_latent) * 0.1)
            self.fc_node_features = nn.Identity()
        else:
            self.static_node_features = nn.Parameter(node_emb, requires_grad=False)
            self.d_node = self.static_node_features.shape[-1]
            self.fc_node_features = nn.Linear(self.d_node, self.d_latent)
        
        self.cells = nn.ModuleList([
            nn.LSTMCell(n_feats if i == 0 else self.d_latent, self.d_latent)
            for i in range(self.n_layers)
        ])
        
        self.readout = nn.Sequential(
            nn.Linear(self.d_latent, self.d_latent), nn.ReLU(),
            nn.Linear(self.d_latent, n_feats)
        )
        
        self.enc_in = None

    def _init_states(
        self, X_0: torch.Tensor, H_0: Optional[torch.Tensor]
    ):
        """
        Trả về list (h_list, c_list) cho từng layer.
        - Nếu H_0 cung cấp (N, d), đặt h_top = H_0; ngược lại nếu enc_in != None thì h_top = enc(X_0); còn lại h=0.
        - c luôn = 0.
        """
        device = X_0.device
        dtype = X_0.dtype
        N = X_0.shape[0]

        h_list = []
        c_list = []
        for _ in range(self.n_layers):
            h_list.append(self.node_features.clone())
            c_list.append(self.node_features.clone())

        if H_0 is not None:
            h_list[-1] = H_0.to(device=device, dtype=dtype)
        elif self.enc_in is not None:
            h_list[-1] = self.enc_in(X_0)

        return h_list, c_list

    def _step(self, x_t: torch.Tensor, h_list, c_list):
        """
        Chạy 1 bước LSTM qua tất cả các layer.
        Input layer nhận x_t; các layer sau nhận h của layer trước ở cùng time-step.
        Trả về (h_list_new, c_list_new, h_top).
        """
        inp = x_t
        new_h, new_c = [], []
        for l, cell in enumerate(self.cells):
            h_t, c_t = cell(inp, (h_list[l], c_list[l]))
            new_h.append(h_t)
            new_c.append(c_t)
            inp = h_t
        h_top = new_h[-1]
        if self.training and self.dropout > 0:
            h_top = F.dropout(h_top, p=self.dropout, training=True)
        return new_h, new_c, h_top

    def forecast(self, X, H_0=None, horizon=0):
        y, *_ = self.forward(X, H_0, horizon)
        return y

    def forward(self, X: torch.Tensor,
                H_0: Optional[torch.Tensor] = None,
                horizon: int = 0) -> Tuple[torch.Tensor, torch.Tensor]:
        """Run deterministic inference.
        Args:
            X: (N, T, F) input features at each t
            H_0: (N, d) optional initial latent state (default zeros)
            Atilde, L: if precomputed; otherwise build from X
        Returns:
            y_pred: (N, T-1) predicted next-day log-return for each node
            H_all: (N, T, d) latent states (including H_0 as first)
        """
        device = self.args.device
        X = X.to(next(self.parameters()).device)
        n_nodes, n_steps, n_feats = X.shape
        d_latent = self.d_latent
        
        self.node_features = self.fc_node_features(self.static_node_features)

        X_0 = X[:, 0, :] # (N, F)
        h_list, c_list = self._init_states(X_0, H_0)

        H_all = torch.zeros(n_nodes, n_steps + max(horizon, 0), d_latent, device=device, dtype=X.dtype)
        r_all = torch.zeros(n_nodes, n_steps - 1 + max(horizon, 0), n_feats, device=device, dtype=X.dtype)
        y_all = torch.zeros(n_nodes, n_steps - 1 + max(horizon, 0), n_feats, device=device, dtype=X.dtype)


        H_all[:, 0] = h_list[-1]
        for t in range(n_steps - 1):
            x_t = X[:, t, :]  # (N, F)
            h_list, c_list, h_top = self._step(x_t, h_list, c_list)
            r_t = self.readout(h_top)
            y_t = x_t + r_t

            H_all[:, t + 1] = h_top
            r_all[:, t] = r_t
            y_all[:, t] = y_t

        if horizon > 0:
            x_t = X[:, -1, :]
            for s in range(horizon):
                h_list, c_list, h_top = self._step(x_t, h_list, c_list)
                r_t = self.readout(h_top)
                y_t = x_t + r_t

                idx = (n_steps - 1) + s
                y_all[:, idx] = y_t
                r_all[:, idx] = r_t
                H_all[:, n_steps + s] = h_top

                x_t = y_t

        return y_all, r_all, H_all

    def forward_loss(
        self,
        X: torch.Tensor,
        H_0: Optional[torch.Tensor] = None,
        horizon: int = 0,
    ):
        """
        Tính loss dự báo one-step + rollout horizon (autoregressive) trên chuỗi đầu vào.

        Args:
            X: (N, T, F) chuỗi gốc (chứa full ground-truth đến T-1)
            H_0: (N, d) latent init (tuỳ chọn)
            horizon: số bước rollout ngoài quan sát cuối cùng
                    (nếu >0, ta cắt input để tránh nhìn thấy tương lai)
            reduction: 'mean' | 'sum' | 'none' cho F.mse_loss

        Returns:
            loss: scalar tensor
            y_pred: (N, T-1, F) dự báo X_{t+1} cho toàn bộ t=0..T-2
        """
        device = next(self.parameters()).device
        X = X.to(device)
        N, T, Fdim = X.shape

        if horizon < 0:
            raise ValueError("horizon must be >= 0")
        if horizon >= T:
            raise ValueError(f"horizon={horizon} must be < sequence length T={T}")

        # Cắt input nếu rollout > 0 để giữ đúng số target (T-1)
        X_in = X[:, : T - horizon, :] if horizon > 0 else X

        # forward() trả (N, (T-horizon)-1 + horizon, F) = (N, T-1, F)
        y_ar, _, _ = self.forward(X_in, H_0=H_0, horizon=horizon)

        # Ground truth luôn là X_{1:T}
        target = X[:, 1:, :]  # (N, T-1, F)
        
        err_t = (y_ar - target).abs().mean(dim=-1).mean(dim=0)
        
        decay = 0.9
        coef_pre = 1.0
        coef_roll = 1.0
        
        len_pre = T - horizon - 1
        loss_pre = torch.tensor(0.0, device=device, dtype=err_t.dtype)
        loss_roll = torch.tensor(0.0, device=device, dtype=err_t.dtype)

        if len_pre > 0:
            idx = torch.arange(len_pre - 1, -1, -1, device=device, dtype=err_t.dtype)
            w = decay ** idx
            loss_pre = (w * err_t[:len_pre]).sum() / (w.sum() + 1e-12)

        if horizon > 0:
            # MSE đều cho đoạn rollout (chiều dài = horizon)
            loss_roll = err_t[len_pre:].mean()

        loss = coef_pre * loss_pre + coef_roll * loss_roll
        
        return loss

# Eval & Train

In [5]:
from sklearn.metrics import *

def eval_ensemble(args, model, training_data_np, testing_data_np, device='cuda', seq_lens=[64, 96, 128], verbose=False):

    used_features = [0, 1, 2, 3, 4, 5]
    training_data_np = np.log1p(training_data_np[:, :, used_features])
    testing_data_np = np.log1p(testing_data_np[:, :, used_features])
    
    labels = torch.tensor(testing_data_np).float().to(device)
    n_nodes, horizon, n_feats = labels.shape
    y_preds = np.zeros((len(seq_lens), n_nodes, horizon, n_feats-1)) # Bỏ Vol
    model.eval().to(device)
    for i, seq_len in enumerate(seq_lens):
        batch = torch.tensor(training_data_np[:, -seq_len:]).float().to(device)
        with torch.no_grad():
            y_all = model.forecast(batch, horizon=horizon)
        y_preds[i] = y_all[:, -horizon:, :n_feats-1].detach().cpu().numpy() # OHLC + Adj Close
    y_gt = testing_data_np[:, :, :n_feats-1].reshape(n_nodes, -1) # OHLC + Adj Close
    y_pred = y_preds.mean(axis=0).reshape(n_nodes, -1)
    
    if verbose:
        print("Max var:", np.var(np.expm1(y_preds), axis=0).max())
        print("Mean var:", np.var(np.expm1(y_preds), axis=0).mean())
    return {
        'rmse': root_mean_squared_error(y_gt, y_pred), 
        'raw_rmse': root_mean_squared_error(np.expm1(y_gt), np.expm1(y_pred)), 
        'mae': mean_absolute_error(y_gt, y_pred), 
        'raw_mae': mean_absolute_error(np.expm1(y_gt), np.expm1(y_pred)), 
        'r2': r2_score(y_gt.ravel(), y_pred.ravel()),
        'raw_r2': r2_score(np.expm1(y_gt).ravel(), np.expm1(y_pred).ravel()), 
    }

def eval(args, model, training_data_np, testing_data_np, device='cuda'):
    return eval_ensemble(args, model, training_data_np, testing_data_np, device, [args.seq_len])

In [6]:
from tqdm import tqdm
import time

class AverageMeter(object):
    """Computes and stores the average and current value
       Imported from https://github.com/pytorch/examples/blob/master/imagenet/main.py#L247-L262
    """

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        if self.avg == 0:
            self.avg = val
            return
        self.avg = 0.95 * self.avg + 0.05 * val

import copy
def train(args, train_loader, model, optimizer, scheduler, training_data_np, testing_data_np):
    # if args.amp:
    #     from apex import amp
    global best_loss, best_model
    test_losses = []
    end = time.time()

    best_model = copy.deepcopy(model)
    step = 0
    for epoch in range(args.epochs):
        batch_time = AverageMeter()
        data_time = AverageMeter()
        losses = AverageMeter()
        stats01 = AverageMeter()
        stats02 = AverageMeter()
        p_bar = tqdm(train_loader)
        for batch_idx, samples in enumerate(p_bar):
            step += 1
            model.train().to(args.device)
          
            samples = samples[0].float().to(args.device)
            data_time.update(time.time() - end)

            loss = model.forward_loss(samples, horizon=args.horizon)
            
            loss.backward()
            
            max_norm = 5.0
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm).item()
            
            optimizer.step()
            scheduler.step()

            losses.update(loss.item())
            stats01.update(0.0)
            # stats01.update(power_iteration_lmax_sym(poly_laplacian(model.L, F.softplus(model.kappa))))

            batch_time.update(time.time() - end)
            end = time.time()
            # mask_probs.update(mask.mean().item())
            p_bar.set_description(
                "Ep: {epoch}/{epochs:3}. LR: {lr:.3e}. "
                "Loss: {loss:.4f}. Stats01: {stats01:.4f}".format(
                epoch=epoch + 1,
                epochs=args.epochs,
                lr=scheduler.get_last_lr()[0],
                data=data_time.avg,
                bt=batch_time.avg,
                loss=losses.avg,
                stats01=stats01.avg,
            ))
            p_bar.update()
            
            if (step + 1) % args.eval_steps == 0:
                test_model = model

                test_metrics = eval_ensemble(args, test_model, training_data_np, testing_data_np, args.device, [args.seq_len])
                print(test_metrics)
                test_loss = test_metrics['mae']

                is_best = test_loss < best_loss
                if test_loss < best_loss:
                    best_loss = test_loss
                    best_model = copy.deepcopy(test_model)


                test_losses.append(test_loss)
                print('Best loss: {:.3f}'.format(best_loss))
                print('Mean loss: {:.3f}\n'.format(
                    np.mean(test_losses[-20:])))


In [7]:
import copy
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR
import math

def get_cosine_schedule_with_warmup(optimizer,
                                    num_warmup_steps,
                                    num_training_steps,
                                    num_cycles=7./16.,
                                    last_epoch=-1):
    def _lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        no_progress = float(current_step - num_warmup_steps) / \
            float(max(1, num_training_steps - num_warmup_steps))
        return max(0., math.cos(math.pi * num_cycles * no_progress))

    return LambdaLR(optimizer, _lr_lambda, last_epoch)

# Training

In [8]:
import os
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd


def manual_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    # if you are suing GPU
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


def get_model_size(model):
	total_size = sum(param.numel() for param in model.parameters() if param.requires_grad)
	return total_size / 1e6

class Config:
    # Training
    epochs = 5
    eval_steps = 200
    lr = 1e-4
    wd = 1e-3
    warmup = 0
    
    n_nodes = 428
    n_feats = 6
    
    # Prediction
    seq_len = 96
    horizon = 32
    
    # LSTM
    d_node: int = None
    d_latent: int = 256
    device: str = "cuda"
    n_layers = 2
    dropout = 0.0
    seed = 42

args = Config()
manual_seed(args.seed)


In [9]:
node_emb_path="../input/static_node_emb.npy"
node_emb_matrix = torch.tensor(np.load(node_emb_path), dtype=torch.float32)


def train_fold(fold_idx):
    global training_data_np, testing_data_np
    used_features = [0, 1, 2, 3, 4, 5]
    training_data_np = train_folds[fold_idx][0][:, :, used_features]
    testing_data_np = train_folds[fold_idx][1][:, :, used_features]
    transformed_training_data_np = np.log1p(training_data_np)
    transformed_testing_data_np = np.log1p(testing_data_np)
    train_loader = DataLoader(SimpleStockDataset(transformed_training_data_np, args.seq_len + args.horizon), batch_size=1, shuffle=True)
    
    manual_seed(args.seed + fold_idx)
        
    model = MyRNN(args.n_nodes, args.n_feats, args, node_emb=node_emb_matrix)

    from torch.optim import AdamW

    optimizer = AdamW(model.parameters(), lr=args.lr, weight_decay=args.wd)
    total_steps = args.epochs * len(train_loader)
    scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=args.warmup,
        num_training_steps=total_steps,
    )
    
    print(f'Model size: {get_model_size(model) * 1e3:.2f}K')
    
    # Sanity check
    print(eval_ensemble(args, model, training_data_np, testing_data_np, args.device, [2, 4]))
    
    global best_loss, best_model
    best_loss = 9999
    best_model = copy.deepcopy(model)

    train(args, train_loader, model, optimizer, scheduler, training_data_np, testing_data_np)
    
    return best_model, best_loss

In [10]:
best_model, best_loss = train_fold(3)

Model size: 868.87K
{'rmse': 0.526685557191528, 'raw_rmse': 358.97996240197466, 'mae': 0.52032424938255, 'raw_mae': 140.37756601242643, 'r2': 0.5457457876254339, 'raw_r2': 0.2382510587744412}


Ep: 1/  5. LR: 9.999e-05. Loss: 0.1968. Stats01: 0.0000:   6%|▌         | 219/3529 [00:51<18:43,  2.95it/s]

{'rmse': 0.07011977558370015, 'raw_rmse': 37.02251639268137, 'mae': 0.05099499445774277, 'raw_mae': 11.628148036166866, 'r2': 0.9934892686186625, 'raw_r2': 0.9935715864602711}
Best loss: 0.051
Mean loss: 0.051



Ep: 1/  5. LR: 9.995e-05. Loss: 0.1922. Stats01: 0.0000:  12%|█▏        | 427/3529 [01:58<17:20,  2.98it/s]

{'rmse': 0.07045433135045637, 'raw_rmse': 36.775431247762945, 'mae': 0.05215796826471413, 'raw_mae': 12.469212926188277, 'r2': 0.993728828773165, 'raw_r2': 0.9933478591116139}
Best loss: 0.051
Mean loss: 0.052



...

Ep: 5/  5. LR: 2.138e-05. Loss: 0.1478. Stats01: 0.0000:  95%|█████████▌| 3365/3529 [10:55<00:36,  4.44it/s]

{'rmse': 0.06115983806344767, 'raw_rmse': 23.324419924046822, 'mae': 0.04066168403942654, 'raw_mae': 8.990140150823585, 'r2': 0.995508665243521, 'raw_r2': 0.9977891107932272}
Best loss: 0.039
Mean loss: 0.040



Ep: 5/  5. LR: 1.985e-05. Loss: 0.1485. Stats01: 0.0000: : 3567it [11:35,  4.44it/s]                        

{'rmse': 0.06056229420831543, 'raw_rmse': 23.247154831373056, 'mae': 0.039937126165006664, 'raw_mae': 8.896663438596937, 'r2': 0.9956000246659883, 'raw_r2': 0.9978002346163533}
Best loss: 0.039
Mean loss: 0.040



Ep: 5/  5. LR: 1.951e-05. Loss: 0.1463. Stats01: 0.0000: 100%|██████████| 3529/3529 [11:44<00:00,  5.01it/s]


In [11]:
best_loss

0.039226465788304486

In [12]:
data, labels = train_folds[2][0], train_folds[2][1]
eval_ensemble(args, best_model, data, labels, args.device, [args.seq_len])

{'rmse': 0.0472313437872839,
 'raw_rmse': 17.395436049034412,
 'mae': 0.03443736801579683,
 'raw_mae': 7.459272183469157,
 'r2': 0.997348623413033,
 'raw_r2': 0.9985920600647545}

In [13]:
data, labels = train_folds[3][0], train_folds[3][1]
eval_ensemble(args, best_model, data, labels, args.device, [args.seq_len])

{'rmse': 0.06024467747250315,
 'raw_rmse': 23.794086696975537,
 'mae': 0.039226465788304486,
 'raw_mae': 9.023921827062413,
 'r2': 0.9956377511923159,
 'raw_r2': 0.9976974200305981}

In [14]:
data, labels = test_fold[0], test_fold[1]
eval_ensemble(args, best_model, data, labels, args.device, [args.seq_len])

{'rmse': 0.05925776160656928,
 'raw_rmse': 29.420537020318868,
 'mae': 0.04265291584034904,
 'raw_mae': 10.661486556444133,
 'r2': 0.9959974936382877,
 'raw_r2': 0.9968750435953792}

In [15]:
data, labels = test_fold[0], test_fold[1]
eval_ensemble(args, best_model, data, labels, args.device, [32, 64, 96])

{'rmse': 0.059266801908728205,
 'raw_rmse': 29.375503162923227,
 'mae': 0.042657423766604745,
 'raw_mae': 10.660737938296819,
 'r2': 0.9959959232576772,
 'raw_r2': 0.996883588096744}

In [16]:
data, labels = test_fold[0], test_fold[1]
eval_ensemble(args, best_model, data, labels, args.device, [64, 96, 128])

{'rmse': 0.05925928017253247,
 'raw_rmse': 29.410658576827814,
 'mae': 0.04265375605845798,
 'raw_mae': 10.66109554258588,
 'r2': 0.9959972308138646,
 'raw_r2': 0.996876998234142}

In [17]:
data, labels = test_fold[0], test_fold[1]
model = copy.deepcopy(best_model)
used_features = [0, 1, 2, 3, 4, 5]
data = np.log1p(data[:, :, used_features])
labels = np.log1p(labels[:, :, used_features])

seq_lens = [64, 96, 128]

labels = torch.tensor(labels).float().to(args.device)
n_nodes, horizon, n_feats = labels.shape
y_preds = np.zeros((len(seq_lens), n_nodes, horizon, n_feats-1)) # Bỏ Vol
model.eval().to(args.device)
for i, seq_len in enumerate(seq_lens):
    batch = torch.tensor(data[:, -seq_len:]).float().to(args.device)
    with torch.no_grad():
        y_all = model.forecast(batch, horizon=horizon)
    y_preds[i] = y_all[:, -horizon:, :n_feats-1].detach().cpu().numpy() # OHLC + Adj Close
y_gt = labels[:, :, :n_feats-1].reshape(n_nodes, -1).detach().cpu().numpy() # OHLC + Adj Close
y_pred = y_preds.mean(axis=0).reshape(n_nodes, -1)


In [18]:
mean_absolute_error(np.expm1(y_gt), np.expm1(y_pred))

10.661093666877838